# Build an Agentic RAG Service

Setup an agent service that can interact with a tool service (containing RAG tools over annual reports).

In this notebook, we:
- Setup our indexes and query engine tools
- Define our multi-agent framework
  - A message queue.
  - An agentic orchestrator.
  - A tools service containing our query engine tools. This will act as a remote executor for tools
  - Define meta-tools for our agents. These will make calls to the tools service instead of executing directly
  - Our agent services. These wrap existing llama-index agents
  - Put all this into a local launcher, to simulate one task passing through the system at a time.

In [1]:
import nest_asyncio

nest_asyncio.apply()

import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai_api_key = os.getenv('OPENAI_API_KEY')

## Load Data

In [2]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-07-26 13:20:56--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.05s   

2024-07-26 13:20:56 (39.4 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-07-26 13:20:56--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connec

In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [4]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/uber")
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [5]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [6]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=10)
uber_engine = uber_index.as_query_engine(similarity_top_k=10)

In [7]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup Agents

Now that we've defined the query tools, we can wrap these under a `ToolService`.

In [9]:
from llama_agents import (
    AgentService,
    ToolService,
    LocalLauncher,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.llms.openai import OpenAI


# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=OpenAI(model="gpt-4o-mini")),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=query_engine_tools,
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in query_engine_tools
]


# define Agent and agent service
worker1 = FunctionCallingAgentWorker.from_tools(
    meta_tools,
    llm=OpenAI(),
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Used to answer questions over Uber and Lyft 10K documents",
    service_name="uber_lyft_10k_analyst_agent",
)

## Launch agent 

With our services, orchestrator, control plane, and message queue defined, we can test our llama-agents network by passing in single messages, and observing the results.

This is an excellent way to test, iterate, and debug your llama-agents system.

In [10]:
import logging

# change logging level to enable or disable more verbose logging
logging.getLogger("llama_agents").setLevel(logging.INFO)

In [11]:
## Define Launcher
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)

In [12]:
# query_str = "What was Lyft's revenue growth in 2021?"
# gets stuck in a loop, should mostly be called once
query_str = "What are the risk factors for Uber?"
result = launcher.launch_single(query_str)

INFO:llama_agents.message_queues.simple - Consumer AgentService-6f47424c-dd55-4f99-896e-ac0fcc53420a: uber_lyft_10k_analyst_agent has been registered.
INFO:llama_agents.message_queues.simple - Consumer ToolService-2c286acb-646b-4c83-a728-2964a4705d36: default_tool_service has been registered.
INFO:llama_agents.message_queues.simple - Consumer 6dc14556-e3e3-4828-a517-b88dfc4cc256: human has been registered.
INFO:llama_agents.message_queues.simple - Consumer ControlPlaneServer-f223f516-894d-413f-b8fd-a71b631ff898: control_plane has been registered.
INFO:llama_agents.services.agent - uber_lyft_10k_analyst_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.services.agent - Processing initiated.
INFO:llama_agents.services.tool - Processing initiated.
INFO:llama_agents.message_queues.base - Publishing message to 'uber_lyft_10k_analys

In [13]:
print(result)

Uber faces several risk factors, including the adverse effects of the COVID-19 pandemic, potential reclassification of drivers as employees, intense competition in mobility and delivery sectors, the need to lower fares and offer incentives, significant historical losses and uncertainty about profitability, risks to brand reputation, challenges in workplace culture and growth management, operational risks related to insurance and regulatory compliance, and potential impacts from pricing regulations and litigation.


In [14]:
query_str = "What was Lyft's revenue growth in 2021?"
result = launcher.launch_single(query_str)

INFO:llama_agents.message_queues.simple - Consumer AgentService-6f47424c-dd55-4f99-896e-ac0fcc53420a: uber_lyft_10k_analyst_agent has been registered.
INFO:llama_agents.message_queues.simple - Consumer ToolService-2c286acb-646b-4c83-a728-2964a4705d36: default_tool_service has been registered.
INFO:llama_agents.message_queues.simple - Consumer 16da8d64-d32e-41d3-b1ed-b25c3a860293: human has been registered.
INFO:llama_agents.message_queues.simple - Consumer ControlPlaneServer-f223f516-894d-413f-b8fd-a71b631ff898: control_plane has been registered.
INFO:llama_agents.services.agent - uber_lyft_10k_analyst_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.services.agent - Processing initiated.
INFO:llama_agents.services.tool - Processing initiated.
INFO:llama_agents.message_queues.base - Publishing message to 'uber_lyft_10k_analys

In [15]:
print(result)

In 2021, Lyft experienced a revenue growth of 36% compared to the previous year.
